In [1]:
# This file is dedicated to name entity pronounciation task. The idea is to record all the syllables of 
# Devnagari and then play them very smoothly to mimic the pronouciation of the name entity given a true 
# spelling.

In [2]:
# Now let's write all the devnagari syllables.

# ankush, sai, hasibul, sangarsh, nehal, muskan

# a aa i u e o - 6

# k kh g gh - 4
# ch chh j jh - 4
# t th d dh n - 5
# p ph b bh m - 5

# y r l w s sh h - 7

# total - 

# So let's say this much syllable is enough.

6 + 4 + 4 + 5 + 5 + 7

31

In [2]:
# Now getting the sounds. 
# Importing necessary packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import tensorflow as tf
import librosa
from IPython.display import Audio
import random
import soundfile as sf
import pyrubberband


2022-02-09 15:30:33.736068: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-09 15:30:33.736096: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
# Now we need a long audio file that will contain all the words distinctly. 

# Let's do a fine cropping program based on amplitude. 

# Importing the audio.
ar, sr = librosa.load('data/system_2/ankush.wav')

# Speed shifting
ar = pyrubberband.pyrb.time_stretch(ar, sr, 2, rbargs=None)

print(ar.shape)
print(sr)

Audio(data=ar, rate=sr)


(28160,)
22050


In [5]:
intervals = librosa.effects.split(ar, top_db=16)
print(intervals)
print(len(intervals))

all_words = np.array([])

# Creating arrays for each words
for i in range(len(intervals)):
    inst_interval = intervals[i]
    inst_audio = ar[inst_interval[0] : inst_interval[1]]
    
    # An extra truncation layer is needed for consonants recordings as it should be just a small part of the 
    # pronunciation.
#     if i >= 0:
#     inst_audio = inst_audio[0:2812]
    
    # Only use this when you are saving audio clips.
    sf.write('data/system_3/words/{}.wav'.format(i+0), inst_audio, sr, 'PCM_24')

    # Concatinating each words.
    all_words = np.concatenate([all_words, inst_audio])

Audio(all_words, rate=sr)


[[  7168  16384]
 [ 23552  32768]
 [ 40960  50176]
 [ 57344  65536]
 [ 73728  82432]
 [ 90624  99328]
 [108032 116736]
 [127488 135680]
 [143872 153600]
 [160768 171008]
 [181248 190464]
 [198656 208896]
 [216576 227840]
 [233984 246272]
 [256512 265216]
 [274944 282624]
 [291328 300544]
 [308736 319488]
 [326656 336384]
 [346624 355328]
 [366080 373760]
 [382464 391680]
 [400896 411136]
 [419328 427520]
 [436736 445952]
 [455168 464896]
 [472576 482304]
 [491008 500224]
 [510464 518144]
 [526848 537088]
 [546304 553472]]
31


In [13]:
# Now we have all the recording saved.
# Now we need to create some index for the vocabulary.

# a aa i u e o 
# k kh g gh 
# ch chh j jh 
# t th d dh n
# p ph b bh m
# y r l w s sh h

syllable_index_dict = {
    'a': 0,
    'aa': 1,
    'i': 2,
    'u': 3,
    'e': 4,
    'o': 5,
    'k': 6,
    'kh': 7,
    'g': 8,
    'gh': 9,
    'ch': 10,
    'chh': 11,
    'j': 12,
    'jh': 13,
    't': 14,
    'th': 15,
    'd': 16,
    'dh': 17,
    'n': 18,
    'p': 19,
    'ph': 20,
    'b': 21,
    'bh': 22,
    'm': 23,
    'y': 24,
    'r': 25,
    'l': 26,
    'w': 27,
    's': 28,
    'sh': 29,
    'h': 30,
}

# Now we have our word_index dict ready. 

In [7]:
# Let's import all of our recordings. 
file_list = os.listdir('data/system_3/words')
print(len(file_list))
print(file_list)


31
['25.wav', '20.wav', '8.wav', '4.wav', '3.wav', '0.wav', '13.wav', '28.wav', '23.wav', '15.wav', '17.wav', '6.wav', '19.wav', '22.wav', '24.wav', '30.wav', '16.wav', '1.wav', '2.wav', '12.wav', '29.wav', '26.wav', '18.wav', '10.wav', '27.wav', '11.wav', '7.wav', '14.wav', '9.wav', '21.wav', '5.wav']


In [8]:
index_ar_dict = {}

for file_name in file_list:
    inst_arr, inst_sr = librosa.load('data/system_3/words/{}'.format(file_name))
    index_ar_dict[file_name.split('.')[0]] = inst_arr
    
display(index_ar_dict)


{'25': array([ 0.00054932, -0.00733948, -0.00453186, ...,  0.02391052,
         0.02256775,  0.01951599], dtype=float32),
 '20': array([ 0.00793457,  0.02590942, -0.01716614, ..., -0.05064392,
        -0.05708313, -0.06159973], dtype=float32),
 '8': array([ 0.00907898,  0.00801086,  0.01144409, ..., -0.05360413,
        -0.05360413, -0.05569458], dtype=float32),
 '4': array([0.05088806, 0.05516052, 0.05532837, ..., 0.02481079, 0.02516174,
        0.02536011], dtype=float32),
 '3': array([-0.0572052 , -0.04949951, -0.04910278, ..., -0.02883911,
        -0.03395081, -0.03649902], dtype=float32),
 '0': array([ 0.01263428,  0.01332092,  0.01647949, ..., -0.04389954,
        -0.04193115, -0.0324707 ], dtype=float32),
 '13': array([-0.04307556, -0.04544067, -0.04415894, ...,  0.04327393,
         0.04627991,  0.0401001 ], dtype=float32),
 '28': array([ 0.0271759 , -0.00617981, -0.00086975, ...,  0.02778625,
         0.03256226,  0.03738403], dtype=float32),
 '23': array([ 0.0005188 ,  0.0010

In [9]:
# Let's play a file from the dict
word = index_ar_dict['12']
Audio(word, rate=sr)


In [10]:
%%time
# Let's construct some sentences
# sentence = 'hamare lon seva me ap _ ka swagat hae . . ' \
# 'd m i faeneans se jankari ke liye _ ek _ dabaen . . ' + \
# 'grahak seva pratinidhi me swagat hae . . ' + \
# 'd m i faeneans se _ lon ke liye _ do _ dabaen . ' + \
# 'pratinidhi sambandhit '

# ankush
sentence = 'a n k u sh'



sentence_ar = sentence.split(' ')
print(sentence_ar)

numeric_ar = []

for word in sentence_ar:
    inst_number = syllable_index_dict[word]
    numeric_ar.append(str(inst_number))
    
print(numeric_ar)


['a', 'n', 'k', 'u', 'sh']
['0', '18', '6', '3', '29']
CPU times: user 548 µs, sys: 195 µs, total: 743 µs
Wall time: 538 µs


In [11]:
%%time
# Re-assembling the words.
final_voice = np.array([])

for number in numeric_ar:
    inst_audio_ar = index_ar_dict[number]
    final_voice = np.concatenate([final_voice, inst_audio_ar])


# Speed shifting
# final_voice = pyrubberband.pyrb.time_stretch(final_voice, sr, 1.0, rbargs=None)

# # Pitch shifting
# final_voice = pyrubberband.pyrb.pitch_shift(final_voice, sr, 1, rbargs=None)

# # Volume shifting
# final_voice = final_voice - 0
    
# Write the final audio.
sf.write('output_{}.wav'.format(str(1)), final_voice, sr, 'PCM_24')

Audio(final_voice, rate=sr)


CPU times: user 5.52 ms, sys: 524 µs, total: 6.04 ms
Wall time: 7.4 ms


In [12]:
# What are some things to be achieved to get the right pronunciation.

# We need very truncated Byanjana Varna.
# We need persistent Swara Varna.
# We need to implement Yuktakshyar. 





